<a href="https://colab.research.google.com/github/AlanRamirez777/Colab/blob/main/Proyecto_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyecto Final: Recordatorio y Envío de Felicitaciones de Cumpleaños
## Alan Omar Ramírez Quiroga

In [2]:
!pip install -q streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 83.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.9 MB/s eta 0:00:00


## Código

In [8]:
%%writefile app.py
import streamlit as st
from datetime import datetime, timedelta, timezone
import time
import csv
import smtplib, ssl
from numpy import random

#--------------------------------------------------------------------------------------------------------------------------------------------------------------

class EmailAdmin: # Esta clase "EmailAdmin" sirve para manejar el envío de datos

  def __init__(self, port = 587, smtp_server = "smtp.gmail.com", sender_email = "alanrcolab@gmail.com", password = "htlh nzio ricx slzc") -> None: #Requiere el puerto, el servidor smtp, correo a usar y la contraseña
    self.port = port
    self.smtp_server = smtp_server
    self.sender_email = sender_email
    self.password = password
    self.context = ssl.create_default_context()

  def SendTo(self, receiver_email:str, message:str): #Envía el correo a un destinatario "receiver_email" con un mensaje "message"
    with smtplib.SMTP(self.smtp_server, self.port) as server:
      server.ehlo()
      server.starttls(context=self.context)
      server.ehlo()
      server.login(self.sender_email, self.password)
      server.sendmail(self.sender_email, receiver_email, message) #Envía el correo
    time.sleep(0.5)

baseEmailAdmin = EmailAdmin() # Creamos la instancia base para el envío de correos

#--------------------------------------------------------------------------------------------------------------------------------------------------------------

class Register: # Clase que administra una instancia de un recordatorio de cumpleaños con diversas configuraciones

  def __init__(self, name: str, month: int, day: int, hour: int, minute: int, email: str, message: str) -> None: # Solicitamos los valores para identificar a la persona y su mensaje; nombre, fecha (month, day. hour, minute), correo electrónico y el mensaje personalizado
    self.id = id
    self.name = name
    self.month = month
    self.day = day
    self.hour = hour
    self.minute = minute
    self.email = email
    self.message = message

  def Apply(self, isRandom: bool, listMsg = []): # Aplica las configuraciones del registro y las aplica para el front-end
    self.isRandom = isRandom
    self.listMsg = listMsg
    with st.expander(self.name):
      self.stEmail = st.write(f"EMAIL: {self.email}")
      self.stDate = st.write(f"FECHA: {self.month}/{self.day} {self.hour}:{self.minute}")
      self.stMessage = st.write(f"MENSAJE: {self.message}")
      self.stColumns = st.columns(5)
      with self.stColumns[0]: self.stC0 = st.empty()
      with self.stColumns[1]: self.stC1 = st.empty()
      with self.stColumns[2]: self.stC2 = st.empty()
      with self.stColumns[3]: self.stC3 = st.empty()
      with self.stColumns[4]: self.stC4 = st.empty()

  def Step(self): # Función ejecutada luego de aplicar, representa un momento para actualizar los datos del contador descendente
    self.stC0.write("Tiempo restante:")
    now = datetime.now(timezone(timedelta(hours=0)))
    result = None
    if now > datetime(now.year, self.month, self.day, self.hour, self.minute, tzinfo=timezone(timedelta(hours=-6))):
      result = datetime(now.year+1, self.month, self.day, self.hour, self.minute, tzinfo=timezone(timedelta(hours=-6))) - now
    else:
      result = datetime(now.year, self.month, self.day, self.hour, self.minute, tzinfo=timezone(timedelta(hours=-6))) - now

    self.stC1.metric("Días", abs(result.days))
    self.stC2.metric("Horas", abs(int(result.seconds/(60*60))))
    self.stC3.metric("Minutos", abs(int(result.seconds/(60))%60))
    self.stC4.metric("Segundos", abs(result.seconds%60))

    if result.total_seconds() <= 0.45:
      baseEmailAdmin.SendTo(self.email, self.message if not self.isRandom else self.listMsg[random.randint(0, len(self.listMsg))])
      st.success(f"Mensaje enviado a {self.name}")

#--------------------------------------------------------------------------------------------------------------------------------------------------------------

class Updatable: # Clase simplificada, la cual su único propósito es la ejecución de funciones en segundo plano

  def __init__(self, time_update=0.1) -> None: # Se crea la instancia y se dá un tiempo de actualización
    self.time_update = time_update
    self.list_events = []

  def Update(self): # Inicia la iteración de sí mismo para ejecutar las funciones
    for i in self.list_events:
      i()
    time.sleep(self.time_update)
    self.Update()

  def AddToUpdate(self, event): # Añade una nueva función a ejecutar
    self.list_events.append(event)

baseUpdatable = Updatable() # Creamos una instancia base de la clase Updatable

#--------------------------------------------------------------------------------------------------------------------------------------------------------------

class MessageAdmin: # Clase que administra y mantiene la lista de mensajes , maneja el guardado y escrito del archivo correspondiente
  FILE_NAME = "messages.csv" # Nombre del archio a manejar

  def __init__(self) -> None: # Al crear la instancia se crea el archivo (si no existe) y obtenemos los datos del archivo
    self.list_messages = []

    try:
      with open(self.FILE_NAME, "x"):
        pass
    except:
      pass

    self.LoadValues()

  def LoadValues(self): # Función para obtener los datos del archivo y los guarda en "self.list_messages". En caso de ser un archivo vacío crea el diseño para guardarlo en archivo .csv
    if not self.check_if_new():
      with open(self.FILE_NAME, mode='r', newline='') as file:
        reader = csv.DictReader(file)
        for row in reader:
          self.list_messages.append(row["messages"])
    else:
      with open(self.FILE_NAME, mode='a', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=['messages'])
        writer.writerow({'messages':'messages'})

  def NewValue(self, value: str): # Agrega un nuevo mensaje a la lista y reescribe todos los valores en el archivo
    self.list_messages.append(value)
    self.WriteAllValues()

  def WriteAllValues(self): # Escribe los valores actuales de la lista en el archivo (self.list_messages)
    with open(self.FILE_NAME, mode='w', newline='') as file:
      writer = csv.DictWriter(file, fieldnames=['messages'])
      writer.writerow({'messages':'messages'})
      for i in self.list_messages:
        writer.writerow({'messages': i})

  def DeleteValue(self, value: str): # Borra un valor de la lista y actualiza todo el archivo
      try:
        self.list_messages.remove(value)
      except:
        pass

  def check_if_new(self) -> bool: # Devuelve True si el archivo es nuevo, False si no
      with open(self.FILE_NAME, mode='r', newline='') as file:
          if file.read() == '':
              return True
          else:
              return False

baseMessageAdmin = MessageAdmin() # Creamos una instancia base de "MessageAdmin"

#--------------------------------------------------------------------------------------------------------------------------------------------------------------

class RegisterAdmin: # Clase que administra y mantiene la lista de registros listta tipo "Register", maneja el guardado y escrito del archivo correspondiente
  FILE_NAME = "register.csv" # Nombre del archivo a manejar

  def __init__(self) -> None: # Al crear la instancia se crea el archivo (si no existe) y obtenemos los datos del archivo

    self.list_register = []

    try:
      with open(self.FILE_NAME, "x"):
        pass
    except:
      pass

    self.LoadValues()

  def LoadValues(self): # Función para obtener los datos del archivo y los guarda en "self.list_register". En caso de ser un archivo vacío crea el diseño para guardarlo en archivo .csv
    if not self.check_if_new():
      with open(self.FILE_NAME, mode='r', newline='') as file:
        reader = csv.DictReader(file)
        for row in reader:
          self.list_register.append(Register(name=row["name"], month=int(row["month"]), day=int(row["day"]), hour=int(row["hour"]), minute=int(row["minute"]), email=row["email"], message=row["message"]))
    else:
      with open(self.FILE_NAME, mode='a', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=['name', 'month', 'day', 'hour', 'minute', 'email', 'message'])
        writer.writerow({'name': 'name', 'month': 'month', 'day': 'day', 'hour': 'hour', 'minute':'minute', 'email':'email', 'message':'message'})

  def NewValue(self, register: Register): # Agrega un nuevo registro a la lista y reescribe todos los valores en el archivo
    self.list_register.append(register)
    self.WriteAllValues()

  def WriteAllValues(self): # Escribe los valores actuales de la lista en el archivo (self.list_register)
    with open(self.FILE_NAME, mode='w', newline='') as file:
      writer = csv.DictWriter(file, fieldnames=['name', 'month', 'day', 'hour', 'minute', 'email', 'message'])
      writer.writerow({'name': 'name', 'month': 'month', 'day': 'day', 'hour': 'hour', 'minute':'minute', 'email':'email', 'message':'message'})
      for i in self.list_register:
        writer.writerow({'name':i.name, 'month':i.month, 'day':i.day, 'hour':i.hour, 'minute':i.minute, 'email':i.email, 'message':i.message})

  def UpdateValue(self, register:Register, newRegister: Register): # Obtiene un el registro a modificar y el registro modificado, reemplaza el valor en la lista y reescribe todo el archivo
    for i in range(len(self.list_register)):
      if register == self.list_register[i]:
        self.list_register.insert(i, newRegister)
        self.list_register.pop(i+1)
        self.WriteAllValues()
        return

  def DeleteValue(self, register: Register): # Borra un valor de la lista y actualiza todo el archivo
    for i in range(len(self.list_register)):
      if register == self.list_register[i]:
        self.list_register.pop(i)
        self.WriteAllValues()
        return

  def check_if_new(self) -> bool: # Devuelve True si el archivo es nuevo, False si no
      with open(self.FILE_NAME, mode='r', newline='') as file:
          if file.read() == '':
              return True
          else:
              return False

  def Apply(self): # Aplica los cambios de la lista de registros listos para mostrar el front-end de cada uno de ellos, implementa la función "Step" de todos a la instancia Actualizable
    for i in self.list_register:
      if i.message != '--Aleatorio--':
        i.Apply(False)
      else:
        i.Apply(True, baseMessageAdmin.list_messages)
      baseUpdatable.AddToUpdate(i.Step)

baseRegisterAdmin = RegisterAdmin() # Creamos una instancia de la clase "RegisterAdmin"

#--------------------------------------------------------------------------------------------------------------------------------------------------------------

#**************************************************************************************************************************************************************

def GetRegister(): # Función del MAIN para mostrar el front-end del ingreso de un nuevo registro
  st.subheader("--Cumpleañero--")
  stName, stEmail = st.columns(2)
  name = stName.text_input("Nombre de la persona:", placeholder="Necesario para añadir/actualizar/borrar una persona")
  email = stEmail.text_input("Correo electrónico:")
  birth = st.columns(4) # Apartado para obtener la fecha de cumpleaños
  month = birth[0].number_input("Mes:", step=1, min_value=1, max_value=12)
  day = birth[1].number_input("Día:", step=1, min_value=1, max_value=31)
  hour = birth[2].number_input("Hora:", step=1, min_value=0, max_value=23)
  minute = birth[3].number_input("Minuto:", step=1, min_value=0, max_value=59)

  msg = st.empty()
  msgValue = ''
  #Definimos en el checkbox si hay mensajes, si los hay muestra una nueva opción aleatoria para un mensaje aleatorio
  if len(baseMessageAdmin.list_messages) == 0:
    msgValue = msg.selectbox("Hola", baseMessageAdmin.list_messages)
    st.warning("Agrega primero un mensaje")
  else:
    a = ['--Aleatorio--']
    a.extend(baseMessageAdmin.list_messages.copy())
    msgValue = msg.selectbox("Mensaje", a)

  buttons = st.columns(3) # Creamos los botones para realizar el CRUD de los registros
  with buttons[0]:
    but0 = st.empty()
  with buttons[1]:
    but1 = st.empty()
  with buttons[2]:
    but2 = st.empty()

  # El primer botón es el de "Añadir", añade un nuevo registro a la instan!ia base de "RegisterAdmin"
  if but0.button("Añadir", use_container_width=True, type='primary'):
    if name!='':
      founded = False
      for i in baseRegisterAdmin.list_register:
        if i.name==name:
          st.warning("Nombre ya ocupado")
          break
      if not founded:
        baseRegisterAdmin.NewValue(Register(name, month, day, hour, minute, email, msgValue))
    else:
      st.warning("Nombre vacío")
  # El segundo botón es "Actualizar", la cual, por medio de su nombre busca y actualiza el registro correspondiente
  if but1.button("Actualizar", use_container_width=True, type='primary'):
      if name!='':
        index = -1
        for i in range(len(baseRegisterAdmin.list_register)):
          if baseRegisterAdmin.list_register[i].name == name:
            index = i
            break
        if index != -1:
          baseRegisterAdmin.UpdateValue(baseRegisterAdmin.list_register[index], Register(name, month, day, hour, minute, email, msgValue))
        else:
          st.warning("Persona no encontrada")
      else:
        st.warning("Nombre vacío")
  # El tercer botón es "Eliminar", elimina el registro que coincida con el nombre dado
  if but2.button("Eliminar", use_container_width=True, type='primary'):
    if name!='' :
      founded = False
      for i in baseRegisterAdmin.list_register:
        if i.name==name:
          baseRegisterAdmin.DeleteValue(i)
          break
      if not founded:
        st.warning("Usuario no encontrado")
    else:
      st.warning("Nombre vacío")

#**************************************************************************************************************************************************************

def GetMessages(): # Método del MAIN para mostrar el front-end del manejo de mensajes
  st.subheader("--Mensaje--")

  columns = st.columns(2) # La primera columnas es para el ingreso de un nuevo mensaje, la segunda columna es para la eliminación de un mensaje
  # Creamos el campo y botón para el ingreso de un nuevo mensaje
  with columns[0]:
    message = st.text_area("Mensaje:")
    if st.button("-Agregar-", use_container_width=True, type='primary'):
      if message != '':
        baseMessageAdmin.NewValue(f"{message}")
      else:
        st.warning("Campo(s) vacío(s)")
  # Crea el selectBox para selecionar el mensaje a eliminar y su respectivo botón
  with columns[1]:
    sel = st.empty()
    butt = st.empty()
    choice = sel.selectbox("A", baseMessageAdmin.list_messages)
    if butt.button("-Eliminar-", use_container_width=True, type='primary'):
      if choice !='':
        baseMessageAdmin.DeleteValue(choice)
      else:
        st.warning("Seleccione un mensaje válido")
    choice = sel.selectbox("Mensaje a borrar:", baseMessageAdmin.list_messages)

#**************************************************************************************************************************************************************

## --MAIN--
# Header introductorio de la página
st.header("SISTEMA AUTOMÁTICO PARA FELICITACIONES")
# Mostramos el front-end del manejo de mensajes
GetMessages()
# Mostramos el front-end del manejo de los registros
GetRegister()
# Usamos la función "Apply" para mostrar el front-end de los registros guardados
baseRegisterAdmin.Apply()
# Iniciamos con la iteración infinita asíncrona de la instancia base Actualizable
baseUpdatable.Update()

# Al apretar un botón el ciclo del --MAIN-- se repite, guardando o eliminando los nuevos valores en el archivo para poder conseguir manejar bien los datos sin perderlos o corromperlos

Overwriting app.py


## Install localtunnel

In [9]:
!npm install localtunnel

⠙⠹⠸⠼⠴
up to date, audited 23 packages in 891ms
⠴
⠴3 packages are looking for funding
⠴  run `npm fund` for details
⠴
2 moderate severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
⠦

## Run streamlit in background

In [10]:
!streamlit run /content/app.py &>/content/logs.txt &

## Expose the port 8501
Then just click in the `url` showed.

A `log.txt`file will be created.

In [11]:
!npx localtunnel --port 8501

⠙your url is: https://nine-parrots-grin.loca.lt
^C
